In [116]:
from __future__ import division 
import pandas as pd
import xgboost as xgb
import numpy as np
import sklearn
import datetime
import scipy as sc
from scipy import stats
import matplotlib.pyplot
import statsmodels as sm
import itertools

#
from numpy import mean, std 
from math import sqrt
import sys

In [8]:
df=pd.read_csv('E:\\datasets\\Natashe\\сравнение для статьи в РПЖ.csv', ';')

In [9]:
df.head()

,варианты ответов,Твое имя,Возраст,"образовательная организация: 1 - школа, 2 - СПО, 3 - ВУЗ","пол: 1 - юноши, 2 - девушки","1 - Творчество, 2 - Относительность взрослости и краткосрочность жизни, 3 - Успех и достижение, 4 - Отказ","1 -отказ, 2 -Противопоставление успешности и неуспешности , 3 - поиск смысла жизни, 4 - Полноценная жизнь и достижения в жизни, 5 - будущее или настоящее, все о времени, 6 - детскость и взрослость",временная удаленность прошлого относительно настоящего,временная удаленность будущего относительно настоящего,глубина конструкта жизни,...,общение,развлечения,дело.1,общение.1,развлечение,дело.2,общение.2,развлечение.1,Unnamed: 19,Unnamed: 20
0,7.0,Мария,23.0,3.0,2.0,NaN,6,10.0,1.0,11.0,...,3.0,0.0,2.0,1.0,0.0,2.0,2.0,0.0,NaN,NaN
1,28.0,Кристина,23.0,3.0,2.0,3,4,6.0,70.0,76.0,...,2.0,0.0,1.0,3.0,0.0,0.0,4.0,0.0,NaN,NaN
2,40.0,Дмитрий,18.0,2.0,1.0,1,5,18.0,NaN,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,62.0,Татьяна,18.0,2.0,2.0,3,6,11.0,7.0,18.0,...,3.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,NaN,NaN
4,72.0,кама,19.0,2.0,2.0,1,1,17.0,2.0,19.0,...,0.0,0.0,2.0,1.0,0.0,2.0,2.0,1.0,NaN,NaN


In [10]:
df.columns

Index(['варианты ответов', 'Твое имя', 'Возраст',
       'образовательная организация: 1 - школа, 2 - СПО, 3 - ВУЗ',
       'пол: 1 - юноши, 2 - девушки',
       '1 - Творчество, 2 - Относительность взрослости и краткосрочность жизни, 3 - Успех и достижение,  4 - Отказ',
       ' 1 -отказ, 2 -Противопоставление успешности и неуспешности , 3 - поиск смысла жизни, 4 - Полноценная жизнь и достижения в жизни, 5 - будущее или настоящее, все о времени, 6 - детскость и взрослость',
       'временная удаленность прошлого относительно настоящего',
       'временная удаленность будущего относительно настоящего',
       'глубина конструкта жизни', 'дело', 'общение', 'развлечения', 'дело.1',
       'общение.1', 'развлечение', 'дело.2', 'общение.2', 'развлечение.1',
       'Unnamed: 19', 'Unnamed: 20'],
      dtype='object')

In [16]:
[print (df[i].unique()) for i in df[['Unnamed: 19', 'Unnamed: 20']]] #посмотрим непонятные столбцы

[nan]
[nan]


[None, None]

In [17]:
df.drop(columns=['Unnamed: 19', 'Unnamed: 20'], inplace=True) #раз они пустые, удалим их

In [21]:
df1=df[['1 - Творчество, 2 - Относительность взрослости и краткосрочность жизни, 3 - Успех и достижение,  4 - Отказ',
       ' 1 -отказ, 2 -Противопоставление успешности и неуспешности , 3 - поиск смысла жизни, 4 - Полноценная жизнь и достижения в жизни, 5 - будущее или настоящее, все о времени, 6 - детскость и взрослость',
       'временная удаленность прошлого относительно настоящего',
       'временная удаленность будущего относительно настоящего',
       'глубина конструкта жизни', 'дело', 'общение', 'развлечения', 'дело.1',
       'общение.1', 'развлечение', 'дело.2', 'общение.2', 'развлечение.1']].copy()
#скопируем нужные для анализа в другой фрейм, старый может еще пригодится, да и места занимает с горошину

In [23]:
df1.dropna(inplace=True) #удалим пустые значения

In [111]:
df1

,"1 - Творчество, 2 - Относительность взрослости и краткосрочность жизни, 3 - Успех и достижение, 4 - Отказ","1 -отказ, 2 -Противопоставление успешности и неуспешности , 3 - поиск смысла жизни, 4 - Полноценная жизнь и достижения в жизни, 5 - будущее или настоящее, все о времени, 6 - детскость и взрослость",временная удаленность прошлого относительно настоящего,временная удаленность будущего относительно настоящего,глубина конструкта жизни,дело,общение,развлечения,дело.1,общение.1,развлечение,дело.2,общение.2,развлечение.1
1,3,4,6.0,70.0,76.0,2.0,2.0,0.0,1.0,3.0,0.0,0.0,4.0,0.0
3,3,6,11.0,7.0,18.0,1.0,3.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0
4,1,1,17.0,2.0,19.0,0.0,0.0,0.0,2.0,1.0,0.0,2.0,2.0,1.0
5,3,4,0.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,2.0
6,4,3,1.0,10.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1388,1,6,21.0,1.0,4.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0
1389,1,4,3.0,1.0,19.0,0.0,3.0,1.0,2.0,1.0,1.0,1.0,4.0,0.0
1390,4,1,23.0,2.0,18.0,3.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,1.0
1391,4,6,23.0,4.0,13.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0


In [58]:
df2=pd.get_dummies(df1, columns=['1 - Творчество, 2 - Относительность взрослости и краткосрочность жизни, 3 - Успех и достижение,  4 - Отказ',
       ' 1 -отказ, 2 -Противопоставление успешности и неуспешности , 3 - поиск смысла жизни, 4 - Полноценная жизнь и достижения в жизни, 5 - будущее или настоящее, все о времени, 6 - детскость и взрослость',])

#подготовим номинальные переменные для анализа

In [59]:
df2.columns

Index(['временная удаленность прошлого относительно настоящего',
       'временная удаленность будущего относительно настоящего',
       'глубина конструкта жизни', 'дело', 'общение', 'развлечения', 'дело.1',
       'общение.1', 'развлечение', 'дело.2', 'общение.2', 'развлечение.1',
       '1 - Творчество, 2 - Относительность взрослости и краткосрочность жизни, 3 - Успех и достижение,  4 - Отказ_1',
       '1 - Творчество, 2 - Относительность взрослости и краткосрочность жизни, 3 - Успех и достижение,  4 - Отказ_2',
       '1 - Творчество, 2 - Относительность взрослости и краткосрочность жизни, 3 - Успех и достижение,  4 - Отказ_3',
       '1 - Творчество, 2 - Относительность взрослости и краткосрочность жизни, 3 - Успех и достижение,  4 - Отказ_4',
       ' 1 -отказ, 2 -Противопоставление успешности и неуспешности , 3 - поиск смысла жизни, 4 - Полноценная жизнь и достижения в жизни, 5 - будущее или настоящее, все о времени, 6 - детскость и взрослость_1',
       ' 1 -отказ, 2 -Противоп

In [60]:
df2.rename(columns={i:i[-7:] for i in df2[['1 - Творчество, 2 - Относительность взрослости и краткосрочность жизни, 3 - Успех и достижение,  4 - Отказ_1',
       '1 - Творчество, 2 - Относительность взрослости и краткосрочность жизни, 3 - Успех и достижение,  4 - Отказ_2',
       '1 - Творчество, 2 - Относительность взрослости и краткосрочность жизни, 3 - Успех и достижение,  4 - Отказ_3',
       '1 - Творчество, 2 - Относительность взрослости и краткосрочность жизни, 3 - Успех и достижение,  4 - Отказ_4']]}, inplace=True)
#Переименуем столбцы для удобства

In [61]:
df2.rename(columns={i:i[-12:] for i in df2[[' 1 -отказ, 2 -Противопоставление успешности и неуспешности , 3 - поиск смысла жизни, 4 - Полноценная жизнь и достижения в жизни, 5 - будущее или настоящее, все о времени, 6 - детскость и взрослость_1',
       ' 1 -отказ, 2 -Противопоставление успешности и неуспешности , 3 - поиск смысла жизни, 4 - Полноценная жизнь и достижения в жизни, 5 - будущее или настоящее, все о времени, 6 - детскость и взрослость_2',
       ' 1 -отказ, 2 -Противопоставление успешности и неуспешности , 3 - поиск смысла жизни, 4 - Полноценная жизнь и достижения в жизни, 5 - будущее или настоящее, все о времени, 6 - детскость и взрослость_3',
       ' 1 -отказ, 2 -Противопоставление успешности и неуспешности , 3 - поиск смысла жизни, 4 - Полноценная жизнь и достижения в жизни, 5 - будущее или настоящее, все о времени, 6 - детскость и взрослость_4',
       ' 1 -отказ, 2 -Противопоставление успешности и неуспешности , 3 - поиск смысла жизни, 4 - Полноценная жизнь и достижения в жизни, 5 - будущее или настоящее, все о времени, 6 - детскость и взрослость_5',
       ' 1 -отказ, 2 -Противопоставление успешности и неуспешности , 3 - поиск смысла жизни, 4 - Полноценная жизнь и достижения в жизни, 5 - будущее или настоящее, все о времени, 6 - детскость и взрослость_6']]}, inplace=True)
#Переименуем столбцы для удобства

In [62]:
df2.columns

Index(['временная удаленность прошлого относительно настоящего',
       'временная удаленность будущего относительно настоящего',
       'глубина конструкта жизни', 'дело', 'общение', 'развлечения', 'дело.1',
       'общение.1', 'развлечение', 'дело.2', 'общение.2', 'развлечение.1',
       'Отказ_1', 'Отказ_2', 'Отказ_3', 'Отказ_4', 'взрослость_1',
       'взрослость_2', 'взрослость_3', 'взрослость_4', 'взрослость_5',
       'взрослость_6'],
      dtype='object')

In [63]:
df2.head()

,временная удаленность прошлого относительно настоящего,временная удаленность будущего относительно настоящего,глубина конструкта жизни,дело,общение,развлечения,дело.1,общение.1,развлечение,дело.2,...,Отказ_1,Отказ_2,Отказ_3,Отказ_4,взрослость_1,взрослость_2,взрослость_3,взрослость_4,взрослость_5,взрослость_6
1,6.0,70.0,76.0,2.0,2.0,0.0,1.0,3.0,0.0,0.0,...,0,0,1,0,0,0,0,1,0,0
3,11.0,7.0,18.0,1.0,3.0,1.0,1.0,2.0,1.0,2.0,...,0,0,1,0,0,0,0,0,0,1
4,17.0,2.0,19.0,0.0,0.0,0.0,2.0,1.0,0.0,2.0,...,1,0,0,0,1,0,0,0,0,0
5,0.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,2.0,...,0,0,1,0,0,0,0,1,0,0
6,1.0,10.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,1,0,0,0


In [103]:
otkaz=list(itertools.combinations(['Отказ_1', 'Отказ_2', 'Отказ_3', 'Отказ_4'], 2))
#подготовим списки переменных для анализа

In [86]:
vrosl=list(itertools.combinations(['взрослость_1',
       'взрослость_2', 'взрослость_3', 'взрослость_4', 'взрослость_5',
       'взрослость_6'], 2))
#подготовим списки переменных для анализа

In [119]:
def cohen_d(x,y):
        return (mean(x) - mean(y)) / sqrt((std(x, ddof=1) ** 2 + std(y, ddof=1) ** 2) / 2.0)
#подготовим формулу расчета Коэффициента Коэна, для расчета величины эффекта

In [127]:
vzrosl1=pd.DataFrame((
    (len(df2.loc[df2[list(i)[0]]==1, ii]), len(df2.loc[df2[list(i)[1]]==1, ii]),
    sc.stats.mannwhitneyu(df2.loc[df2[list(i)[0]]==1, ii], df2.loc[df2[list(i)[1]]==1, ii])[1],
    sc.stats.ttest_ind(df2.loc[df2[list(i)[0]]==1, ii], df2.loc[df2[list(i)[1]]==1, ii])[1],
     cohen_d(df2.loc[df2[list(i)[0]]==1, ii], df2.loc[df2[list(i)[1]]==1, ii]), i, ii)) 
 for ii in ['временная удаленность прошлого относительно настоящего',
       'временная удаленность будущего относительно настоящего',
       'глубина конструкта жизни', 'дело', 'общение', 'развлечения', 'дело.1',
       'общение.1', 'развлечение', 'дело.2', 'общение.2', 'развлечение.1'] for i in vrosl)
vzrosl1.rename(columns={0:'Количество в первой группе', 1:'Количество во второй группе',  2:'p value манна-уитни',\
                        3:'p value t-test', 4:'Д-Коэн', 5: 'Группы G', 6: 'Другая методика'}\
               , inplace=True)
vzrosl1
#Проведем необходимые расчеты и сделаем дата фрейм

,Количество в первой группе,Количество во второй группе,p value манна-уитни,p value t-test,Д-Коэн,Группы G,Другая методика
0,209,182,0.011209,0.153851,-0.146253,"(взрослость_1, взрослость_2)",временная удаленность прошлого относительно на...
1,209,238,0.042279,0.265937,-0.104816,"(взрослость_1, взрослость_3)",временная удаленность прошлого относительно на...
2,209,56,0.187693,0.690795,-0.063269,"(взрослость_1, взрослость_4)",временная удаленность прошлого относительно на...
3,209,208,0.318335,0.859131,0.017399,"(взрослость_1, взрослость_5)",временная удаленность прошлого относительно на...
4,209,108,0.356431,0.941769,-0.008980,"(взрослость_1, взрослость_6)",временная удаленность прошлого относительно на...
...,...,...,...,...,...,...,...
175,238,208,0.299418,0.987696,-0.001460,"(взрослость_3, взрослость_5)",развлечение.1
176,238,108,0.453062,0.557163,-0.065663,"(взрослость_3, взрослость_6)",развлечение.1
177,56,208,0.433400,0.739682,-0.052754,"(взрослость_4, взрослость_5)",развлечение.1
178,56,108,0.474131,0.500412,-0.116260,"(взрослость_4, взрослость_6)",развлечение.1


In [128]:
otkaz1=pd.DataFrame((
    (len(df2.loc[df2[list(i)[0]]==1, ii]), len(df2.loc[df2[list(i)[1]]==1, ii]),
    sc.stats.mannwhitneyu(df2.loc[df2[list(i)[0]]==1, ii], df2.loc[df2[list(i)[1]]==1, ii])[1],
    sc.stats.ttest_ind(df2.loc[df2[list(i)[0]]==1, ii], df2.loc[df2[list(i)[1]]==1, ii])[1],
     cohen_d(df2.loc[df2[list(i)[0]]==1, ii], df2.loc[df2[list(i)[1]]==1, ii]), i, ii)) 
    for ii in ['временная удаленность прошлого относительно настоящего',
       'временная удаленность будущего относительно настоящего',
       'глубина конструкта жизни', 'дело', 'общение', 'развлечения', 'дело.1',
       'общение.1', 'развлечение', 'дело.2', 'общение.2', 'развлечение.1'] for i in otkaz)
otkaz1.rename(columns={0:'Количество в первой группе', 1:'Количество во второй группе',  2:'p value манна-уитни',\
                        3:'p value t-test', 4:'Д-Коэн', 5: 'Группы G', 6: 'Другая методика'}\
               , inplace=True)
otkaz1
#Проведем необходимые расчеты и сделаем дата фрейм

,Количество в первой группе,Количество во второй группе,p value манна-уитни,p value t-test,Д-Коэн,Группы G,Другая методика
0,358,149,0.492420,0.990159,0.001206,"(Отказ_1, Отказ_2)",временная удаленность прошлого относительно на...
1,358,359,0.174300,0.244962,-0.086924,"(Отказ_1, Отказ_3)",временная удаленность прошлого относительно на...
2,358,135,0.197894,0.323204,-0.098640,"(Отказ_1, Отказ_4)",временная удаленность прошлого относительно на...
3,149,359,0.247423,0.381664,-0.088494,"(Отказ_2, Отказ_3)",временная удаленность прошлого относительно на...
4,149,135,0.229891,0.397949,-0.100424,"(Отказ_2, Отказ_4)",временная удаленность прошлого относительно на...
...,...,...,...,...,...,...,...
67,358,359,0.104759,0.239375,0.087949,"(Отказ_1, Отказ_3)",развлечение.1
68,358,135,0.489361,0.880767,-0.014974,"(Отказ_1, Отказ_4)",развлечение.1
69,149,359,0.187435,0.156347,0.132075,"(Отказ_2, Отказ_3)",развлечение.1
70,149,135,0.468915,0.756686,0.036966,"(Отказ_2, Отказ_4)",развлечение.1
